<h1><center>  lab 7 : Probablistic Language Models </center>
    
<img src="https://lena-voita.github.io/resources/lectures/lang_models/evaluation/idea-min.png" width="700">


```Created by Jinnie Shin (jinnie.shin@coe.ufl.edu)```\
```Date: ```

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQmNf86oJnfhpkPA9LnrFnAbfwF2VywPYpB_w&usqp=CAU" align="left" width="70" height="70" align="left">

 ### Required Packages or Dependencies

In [1]:
import pandas as pd
import numpy as np


## **REVIEW**: Dataset

> Using the text_normalizer function we created last time, we will import `essay set 5`
#### Essay Set 5
`Prompt`: Starting with mRNA leaving the nucleus, list and describe four major steps involved in protein synthesis.


| Type of response            | Source dependent response |
|-----------------------------|---------------------------|
| Grade level                 | `10`                       |
| Subject                     | `Biology`                   |
| Total sample size           | `1,795`                |
| Average length of responses | `60 words`                  |
| Score range                 | `0-3`                       |

In [2]:
import pandas as pd
import nltk
nltk.download('punkt')

# 1. open the file and save only the essay set 5 -- call this `data`
# HINT pd.read_excel('NAME')
data = pd.read_excel('training_set_rel3.xlsx')
data = data[data.essay_set==5]

# 2. extract only the following two columns from the dataframe you constructed in step 1
# : "essay", "rater1_domain1"
# give the two columns new names --- "response_full" and "score"

data = data[['essay', 'rater1_domain1']]
data.columns = ['response_full', 'score']

# 2. create a column that saves the word tokenization results -- call this `response`
data['response'] = data.response_full.apply(nltk.word_tokenize)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


<img src="https://i.pinimg.com/736x/2e/aa/7d/2eaa7d5021ca7c3c98bc93b98b9646fe.jpg" align="left" width="70" height="70" align="left">

 ## Task 1: Training & Testing data
>  Q1. We will use the essay sets from the score 3 and 2 categories as the training set  (`training_data`). The `training_data` only containes one column `"response_full"` \
> Q2. Combine all the responses in the `training_data` into one document (`str` object). Include the `<s>` and `<\s>` at the start and the end of each sentence.

> Q3. Create a dataframe called `testing_data`, which only includes the rows with the score categories of 1 and 0. Include the `<s>` and `<\s>` at the start and the end of each sentence.


In [6]:
################################ YOUR CODE HERE for Q1 ###########################
training_data = data.query("score == 2|score == 3") # could use .isin([2,3])
train = training_data["response_full"]
##################################################################################

In [11]:
################################ YOUR CODE HERE for Q2 ###########################

training_corpus = ""

for i in train:
  train_sents = nltk.sent_tokenize(i)
  for i in train_sents:
    train_sents_s = " <s> " + i + " </s> " # spaces help keep them separate

    training_corpus += train_sents_s # same as writing training_corpus = training_corpus + train_sents

print(training_corpus)

##################################################################################

 <s> In this memoir of Narciso Rodriguez, @PERSON3's life, the mood containing it all, was greatful, and showed how his parents gave him love through his whole life. </s>  <s> Both his parents were born and raised in Cuba, and in 1956, they both moved to the United States. </s>  <s> Starting their lives over again, and taking any job they could find. </s>  <s> Then in 1961, @PERSON2, @CAPS1. </s>  <s> was born. </s>  <s> Both Parents raised him with love, and care, and introducing his Cuban background into his life. </s>  <s> As @PERSON2, was telling his story, all readers could tell how greatful he was to have a family like he did. </s>  <s> Also, as he shared his non-– blood related family, and the remembrence on how his parents' life changed by moving to @LOCATION2, he showed the respect he had towards them. </s>  <s> Throughout the excerpt from Home the Blueprints of Our Lives, Narciso Rodriguez, the author, manages to maintain a single overall mood. </s>  <s> In many parts of the 

In [ ]:
################################ YOUR CODE HERE for Q3 ###########################

##################################################################################


## 1. Language Model Construction
### 1.1. N-gram

> Let's explore the function, `nltk.FreqDist()` and create our own that takes bi- & tri-grams

In [12]:
import nltk
example_text = data.response.iloc[0]
print(example_text)
nltk.FreqDist(example_text) # it takes a list of tokens as an input

# how do we create a frequency count for bi-gram and tri-gram information?

['In', 'this', 'memoir', 'of', 'Narciso', 'Rodriguez', ',', '@', 'PERSON3', "'s", 'life', ',', 'the', 'mood', 'containing', 'it', 'all', ',', 'was', 'greatful', ',', 'and', 'showed', 'how', 'his', 'parents', 'gave', 'him', 'love', 'through', 'his', 'whole', 'life', '.', 'Both', 'his', 'parents', 'were', 'born', 'and', 'raised', 'in', 'Cuba', ',', 'and', 'in', '1956', ',', 'they', 'both', 'moved', 'to', 'the', 'United', 'States', '.', 'Starting', 'their', 'lives', 'over', 'again', ',', 'and', 'taking', 'any', 'job', 'they', 'could', 'find', '.', 'Then', 'in', '1961', ',', '@', 'PERSON2', ',', '@', 'CAPS1', '.', 'was', 'born', '.', 'Both', 'Parents', 'raised', 'him', 'with', 'love', ',', 'and', 'care', ',', 'and', 'introducing', 'his', 'Cuban', 'background', 'into', 'his', 'life', '.', 'As', '@', 'PERSON2', ',', 'was', 'telling', 'his', 'story', ',', 'all', 'readers', 'could', 'tell', 'how', 'greatful', 'he', 'was', 'to', 'have', 'a', 'family', 'like', 'he', 'did', '.', 'Also', ',', 'as'

FreqDist({',': 16, 'his': 8, '.': 8, 'and': 7, '@': 5, 'he': 5, 'life': 4, 'the': 4, 'was': 4, 'how': 3, ...})

### 1.2. Language Model Construction
> We will create an n-gram based langauge model, `LM`, which is trained on the given essay set

In [13]:
import math
from itertools import product
class LM(object):
    def __init__(self, train_data, n, laplace=1):
        self.n = n
        self.laplace = laplace
        self.tokens = train_data.split(' ')
        self.vocab  = nltk.FreqDist(self.tokens)
        self.model  = self._create_model()
        self.masks  = list(reversed(list(product((0,1), repeat=n))))

    def _smooth(self):
        vocab_size = len(self.vocab)

        n_grams = nltk.ngrams(self.tokens, self.n)
        n_vocab = nltk.FreqDist(n_grams)

        m_grams = nltk.ngrams(self.tokens, self.n-1)
        m_vocab = nltk.FreqDist(m_grams)

        def smoothed_count(n_gram, n_count):
            m_gram = n_gram[:-1]
            m_count = m_vocab[m_gram]
            return (n_count + self.laplace) / (m_count + self.laplace * vocab_size)

        return { n_gram: smoothed_count(n_gram, count) for n_gram, count in n_vocab.items() }

    def _create_model(self):
        if self.n == 1:
            num_tokens = len(self.tokens)
            return { (unigram,): count / num_tokens for unigram, count in self.vocab.items() }
        else:
            return self._smooth()

    def _convert_oov(self, ngram):
        mask = lambda ngram, bitmask: tuple((token if flag == 1 else "<UNK>" for token,flag in zip(ngram, bitmask)))

        ngram = (ngram,) if type(ngram) is str else ngram
        for possible_known in [mask(ngram, bitmask) for bitmask in self.masks]:
            if possible_known in self.model:
                return possible_known

    def perplexity(self, test_data):
        test_tokens = test_data.split(' ')
        test_ngrams = nltk.ngrams(test_tokens, self.n)
        N = len(test_tokens)

        known_ngrams  = (self._convert_oov(ngram) for ngram in test_ngrams)
        probabilities = [self.model[ngram] for ngram in known_ngrams]

        return math.exp((-1/N) * sum(map(math.log, probabilities))), test_tokens

    def _best_candidate(self, prev, i, without=[]):
        blacklist  = ["<UNK>"] + without
        candidates = ((ngram[-1],prob) for ngram,prob in self.model.items() if ngram[:-1]==prev)
        candidates = filter(lambda candidate: candidate[0] not in blacklist, candidates)
        candidates = sorted(candidates, key=lambda candidate: candidate[1], reverse=True)
        if len(candidates) == 0:
            return ("</s>", 1)
        else:
            return candidates[0 if prev != () and prev[-1] != "<s>" else i]

    def generate_sentences(self, num, min_len=12, max_len=24):
        for i in range(num):
            sent, prob = ["<s>"] * max(1, self.n-1), 1
            while sent[-1] != "</s>":
                prev = () if self.n == 1 else tuple(sent[-(self.n-1):])
                blacklist = sent + (["</s>"] if len(sent) < min_len else [])
                next_token, next_prob = self._best_candidate(prev, i, without=blacklist)
                sent.append(next_token)
                prob *= next_prob

                if len(sent) >= max_len:
                    sent.append("</s>")

            yield ' '.join(sent), -1/math.log(prob)

### 1.3  Sentence Generation with a new Language Model

In [18]:
lm = LM(training_corpus, n = 2, laplace = 1)

## Creates the 10 most probable sentences
print("Generating sentences...")
for sentence, prob in lm.generate_sentences(10):
   print("{} ({:.5f})".format(sentence, prob))


Generating sentences...
<s> The mood created by the author in this memoir is a better life. </s> (0.02279)
<s> In the author in this memoir is a better life. </s> (0.03021)
<s> He is a better life. </s> (0.05191)
<s> This is a better life. </s> (0.05204)
<s> Narciso Rodriguez is a better life. </s> (0.04263)
<s> I think the author in this memoir is a better life. </s> (0.02343)
<s> They came together in the author is a better life. </s> (0.02580)
<s> It is a better life. </s> (0.04802)
<s> @CAPS1 is a better life. </s> (0.04877)
<s> His parents for their love and how his family is a better life. </s> (0.01871)


<img src="https://i.pinimg.com/736x/2e/aa/7d/2eaa7d5021ca7c3c98bc93b98b9646fe.jpg" align="left" width="70" height="70" align="left">

 ## Task 2: Perplexity Score
>  Q1. Generate a new column called `perpleixty` which saves the perplexity score of the `response_full` columns of the `testing_data`. \
( HINT: `lm.perplexity("test sentence here")` and `.apply`)

In [ ]:

################################ YOUR CODE HERE for Q1 ###########################
##################################################################################
